In [ ]:
from spatiallyAdaptiveSingleDimension2 import SpatiallyAdaptiveSingleDimensions2
%matplotlib inline
import matplotlib.pyplot as plt
from sys import path
import numpy as np

#from numpy import *
import scipy as sp

from PerformTestCase import *
from Function import *
from ErrorCalculator import *
import math

# evaluation points for error estimation
def evaluation_points(a, b, d):
    # n_bins =  100 * np.ones(d)
    # bounds = np.repeat([(0,1)], d, axis = 0) # TODO change to actual bounds
    # grid = np.mgrid[[slice(row[0], row[1], n*1j) for row, n in zip(bounds, n_bins)]]
    # return grid.reshape(d,-1).T

    grid = np.meshgrid(*(np.linspace(a[i], b[i], 10) for i in range(len(a))))
    points = list(zip(*(x.flat for x in grid)))

    return points


# shortcut for romberg SASD
def get_single_dim_romberg(a, b, f, reference_solution,
                           grid_version, coefficient_version, force_full_binary_tree_grid,
                           optimized_container_splitting):
    grid_romberg = GlobalRombergGrid(a=a, b=b, modified_basis=False, boundary=True,
                                     grid_version=grid_version,
                                     coefficient_version=coefficient_version,
                                     optimized_container_splitting=optimized_container_splitting)

    operation_romberg = Integration(f=f, grid=grid_romberg, dim=dim, reference_solution=reference_solution)

    return SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_romberg, rebalancing=False,
                                              force_full_binary_tree_grid=force_full_binary_tree_grid)

# Settings
dim = 5
a = np.zeros(dim)
b = np.ones(dim)
max_tol = 8
max_evaluations = 10 ** 5
evaluation_points = evaluation_points(a, b, dim)
print(evaluation_points)

# --- Smooth functions

# coeffs = np.ones(dim)
# midpoint = np.ones(dim) * 0.5
# f = GenzProductPeak(coeffs, midpoint)

# coeffs = np.ones(dim)
# f = GenzCornerPeak(coeffs=coeffs)

# coeffs = np.ones(dim)
# offset = 1
# f = GenzOszillatory(coeffs, offset)

# coeffs = np.ones(dim)
# midpoint = np.ones(dim) * 0.5
# f = GenzGaussian(coeffs, midpoint)

# --- Genz discontinuous

midpoint = np.ones(dim) * 0.5
coefficients = np.array([10**0 * (d+1) for d in range(dim)])

f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# f = GenzDiscontinious2(border=midpoint,coeffs=coefficients)

# Plot
f.plot(np.ones(dim)*a,np.ones(dim)*b)
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# Trapezoidal Grid
grid_trapezoidal = GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)
operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
SASD_trapezoidal = SpatiallyAdaptiveSingleDimensions2(a, b,
                                                            operation=operation_trapezoidal, rebalancing=False,
                                                            force_full_binary_tree_grid=False)

errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# grid for error estimation
grid = TrapezoidalGrid(a, b, boundary=True)

Unit Slices

In [ ]:
r_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                grid_version=RombergGridVersion.UNIT_SLICES,
                                coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                force_full_binary_tree_grid=False,
                                optimized_container_splitting=False)

r_full_binary_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                            grid_version=RombergGridVersion.UNIT_SLICES,
                                            coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                            force_full_binary_tree_grid=True,
                                            optimized_container_splitting=False)

algorithmArray = [
    (SASD_trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid.'),
    (r_unit, 1, 2, errorOperator, 'RombergGrid (Unit).'),
    (r_full_binary_unit, 1, 2, errorOperator, 'RombergGrid (Full, Unit).'),
]
performTestcaseArbitraryDim(f, a, b, algorithmArray,
                            max_tol, dim, 2, grid=grid, evaluation_points=evaluation_points,
                            max_evaluations=max_evaluations,
                            save_with_name="unit_slices"
                            )

Slice Grouping

In [ ]:
r_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                   grid_version=RombergGridVersion.GROUPED_SLICES,
                                   coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                   force_full_binary_tree_grid=False,
                                   optimized_container_splitting=False)

r_full_binary_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                               grid_version=RombergGridVersion.GROUPED_SLICES,
                                               coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                               force_full_binary_tree_grid=True,
                                               optimized_container_splitting=False)

algorithmArray = [
    (SASD_trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid.'),
    (r_grouped, 1, 2, errorOperator, 'RombergGrid (Grouped).'),
    (r_full_binary_grouped, 1, 2, errorOperator, 'RombergGrid (Full, Grouped).'),
]

performTestcaseArbitraryDim(f, a, b, algorithmArray,
                            max_tol, dim, 2, grid=grid, evaluation_points=evaluation_points,
                            max_evaluations=max_evaluations,
                            save_with_name="grouped_slices"
                            )


Optimized slice grouping

In [ ]:
r_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                   grid_version=RombergGridVersion.GROUPED_SLICES,
                                   coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                   force_full_binary_tree_grid=False,
                                   optimized_container_splitting=True)

r_full_binary_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                               grid_version=RombergGridVersion.GROUPED_SLICES,
                                               coefficient_version=ExtrapolationCoefficientVersion.ROMBERG,
                                               force_full_binary_tree_grid=True,
                                               optimized_container_splitting=True)

algorithmArray = [
    (SASD_trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid.'),
    (r_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Grouped, Optimized).'),
    (r_full_binary_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Full, Grouped, Optimized).'),
]

performTestcaseArbitraryDim(f, a, b, algorithmArray,
                            max_tol, dim, 2, grid=grid, evaluation_points=evaluation_points,
                            max_evaluations=max_evaluations,
                            save_with_name="grouped_slices_optimized"
                            )


Slice Grouping Trapezoidal Coefficients

In [ ]:
r_full_binary_slice = get_single_dim_romberg(a, b, f, reference_solution,
                                             grid_version=RombergGridVersion.UNIT_SLICES,
                                             coefficient_version=ExtrapolationCoefficientVersion.TRAPEZOIDAL_SLICE,
                                             force_full_binary_tree_grid=True,
                                             optimized_container_splitting=False)

algorithmArray = [
    (SASD_trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid.'),
    (r_full_binary_slice, 1, 2, errorOperator, 'RombergGrid (Full, Unit, Trapezoidal).'),
]

performTestcaseArbitraryDim(f, a, b, algorithmArray,
                            max_tol, dim, 2, grid=grid, evaluation_points=evaluation_points,
                            max_evaluations=max_evaluations,
                            save_with_name="unit_slices_trapezoidal"
                            )